<a href="https://colab.research.google.com/github/dletterle/Amazon_Vine_Analysis/blob/main/Vine_Review_Analysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import os
# Find the latest version of spark 3.0 from http://www.apache.org/dist/spark/ and enter as the spark version
# For example:
# spark_version = 'spark-3.0.3'
spark_version = 'spark-3.0.3'
os.environ['SPARK_VERSION']=spark_version

# Install Spark and Java
!apt-get update
!apt-get install openjdk-11-jdk-headless -qq > /dev/null
!wget -q http://www.apache.org/dist/spark/$SPARK_VERSION/$SPARK_VERSION-bin-hadoop2.7.tgz
!tar xf $SPARK_VERSION-bin-hadoop2.7.tgz
!pip install -q findspark

# Set Environment Variables
import os
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-11-openjdk-amd64"
os.environ["SPARK_HOME"] = f"/content/{spark_version}-bin-hadoop2.7"

# Start a SparkSession
import findspark
findspark.init()

In [ ]:
# Download the Postgres driver that will allow Spark to interact with Postgres.
!wget https://jdbc.postgresql.org/download/postgresql-42.2.16.jar

In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("M16-Amazon-Challenge").config("spark.driver.extraClassPath","/content/postgresql-42.2.16.jar").getOrCreate()

In [ ]:
from pyspark import SparkFiles
url = "https://s3.amazonaws.com/amazon-reviews-pds/tsv/amazon_reviews_us_Video_Games_v1_00.tsv.gz"
spark.sparkContext.addFile(url)
df = spark.read.option("encoding", "UTF-8").csv(SparkFiles.get("amazon_reviews_us_Video_Games_v1_00.tsv.gz"), sep="\t", header=True, inferSchema=True)
df.show()

In [ ]:
drop_na_df = df.dropna()
drop_na_df.show(

In [ ]:
#recreate the vine_df from first part of challenge
vine_df = drop_na_df.select(["review_id","star_rating","helpful_votes","total_votes","vine","verified_purchase"])
#create df with total votes greater than or equal to 20
filtered_vine_df=vine_df.filter("total_votes>=20")
filtered_vine_df.show()

In [ ]:
helpful_reviews_df = filtered_vine_df.filter("(helpful_votes/total_votes)>=0.5")
helpful_reviews_df.show()

In [ ]:
vine_helpful_reviews_df=helpful_reviews_df.filter("vine=='Y'")
vine_helpful_reviews_df.show()

In [ ]:
not_vine_helpful_reviews_df=helpful_reviews_df.filter("vine=='N'")
not_vine_helpful_reviews_df.show()

In [ ]:
total_num_helpful_reviews = helpful_reviews_df.count()
total_num_helpful_reviews

In [ ]:
total_num_helpful_vine_reviews = vine_helpful_reviews_df.count()
total_num_helpful_vine_reviews

In [ ]:
total_num_helpful_not_vine_reviews = not_vine_helpful_reviews_df.count()
total_num_helpful_not_vine_reviews

In [ ]:
five_star_vine_reviews = vine_helpful_reviews_df.filter("star_rating=='5'").count()
five_star_vine_reviews

In [ ]:
five_star_not_vine_reviews = not_vine_helpful_reviews_df.filter("star_rating=='5'").count()
five_star_not_vine_reviews

In [ ]:
percent_five_star_vine_reviews = (five_star_vine_reviews/total_num_helpful_vine_reviews)*100
percent_five_star_vine_reviews

In [ ]:
percent_five_star_not_vine_reviews = (five_star_not_vine_reviews/total_num_helpful_not_vine_reviews)*100
percent_five_star_not_vine_reviews